<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Diffusion_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# Simple MLP for the denoising function
class DenoiseMLP(nn.Module):
    def __init__(self):
        super(DenoiseMLP, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(2, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        return self.fc(x)

# Define the noise schedule
def noise_schedule(t, beta_start=0.0001, beta_end=0.02):
    return beta_start + t * (beta_end - beta_start)

# Forward and reverse diffusion processes
def forward_diffusion(x, t, noise_schedule):
    beta_t = noise_schedule(t)
    noise = torch.randn_like(x)
    return np.sqrt(1 - beta_t) * x + np.sqrt(beta_t) * noise

def reverse_diffusion(x, t, model, noise_schedule):
    beta_t = noise_schedule(t)
    denoised = model(x)
    return (x - np.sqrt(beta_t) * denoised) / np.sqrt(1 - beta_t)

# Training the model
model = DenoiseMLP()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

for epoch in range(1000):
    t = np.random.rand()
    x = torch.randn(1000, 2)
    x_noisy = forward_diffusion(x, t, noise_schedule)

    optimizer.zero_grad()
    denoised = model(x_noisy)
    loss = criterion(denoised, x)
    loss.backward()
    optimizer.step()

# Generate samples
t = 0.1
x = torch.randn(1000, 2)
for _ in range(100):
    x = reverse_diffusion(x, t, model, noise_schedule)

plt.scatter(x[:, 0].detach().numpy(), x[:, 1].detach().numpy())
plt.show()